In [1]:
import pickle
import numpy as np
import tqdm

In [118]:
arr1 = np.load("long_list.npy")
arr2 = np.load("short_list.npy", allow_pickle=True)

In [119]:
len(arr1), len(arr2)

(10657, 2315)

In [120]:
f = lambda l:"".join([chr(e) for e in l])

In [149]:
comb = np.concatenate([arr1, arr2])
comb = np.array(sorted(comb, key=f))

In [63]:
def comp(l1, l2):
    for i in range(len(l1)):
        if l1[i] < l2[i]:
            return -1
        if l1[i] > l2[i]:
            return 1
    return 0
def is_sorted(l):
    for w1, w2 in zip(l[:-1], l[1:]):
        if comp(w1, w2) != -1:
            return False
    return True

In [157]:
is_sorted(arr1), is_sorted(arr2), is_sorted(comb)

(True, True, True)

In [160]:
np.save("combined.npy", comb)
np.save("short_list.npy", arr2)

In [109]:
def fb(guess, ans):
    d = {i:i for i in range(5) if guess[i] == ans[i]}
    for i in range(5):
        if i in d:
            continue
        for j in range(5):
            if j in d.values():
                continue
            if guess[i] == ans[j]:
                d[i] = j
    out = 0
    for i in range(5):
        if i not in d:
            continue
        elif d[i] == i:
            out += np.power(2, i)
        else:
            out += 32*np.power(2, i)
    return out

#### Examples
Guessing `reeds` when `elder` is the answer. Feedback is `YYYY_`, or

`[[0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0]]`
  
The feedback number is $32+64+128+256 = 480$.

Guessing `float` when `aloft` is the answer. Feedback is `YGGYG`, or

`[[0, 1, 1, 0, 1],
  [1, 0, 0, 1, 0]]`
  
The feedback number is $2+4+16+32+256=310$.

In [163]:
fi = lambda l:list(map(ord, l))

In [166]:
fb(fi("reeds"), fi("elder")), fb(fi("float"), fi("aloft"))

(480, 310)

In [168]:
all_feedbacks = np.zeros((len(arr2), len(comb)))

In [169]:
all_feedbacks.shape

(2315, 12972)

In [173]:
for i in tqdm.trange(len(arr2)):
    ans = arr2[i]
    for j, guess in enumerate(comb):
        all_feedbacks[i][j] = fb(guess, ans)

100%|██████████| 2315/2315 [06:02<00:00,  6.38it/s]


In [179]:
all_feedbacks = all_feedbacks.astype("int64")

In [180]:
all_feedbacks[641][9024] # elder, reeds

480

In [181]:
np.save("feedbacks.npy", all_feedbacks)